In [11]:
import pandas as pd
import numpy as np
import joblib
import os,sys
import benchmark_common as bcommon
import config as cfg
import argparse
import tools.funclib as funclib
from tools.Attention import Attention
from keras.models import load_model
import tools.embedding_esm as esmebd
import time
from pandarallel import pandarallel #  import pandaralle

pandarallel.initialize() #init


INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [12]:
input_fasta = cfg.DATADIR + 'uniprotkb_Escherichia_coli_AND_model_or_2023_07_13.fasta'
output_tsv = cfg.RESULTSDIR + 'ec_resuniprotkb_Escherichia_coli_AND_model_or_2023_07_13.tsv'
topnum=20
mode='h'

In [13]:

# 1. 读入数据
print('step 1: loading data')
input_df = funclib.load_fasta_to_table(input_fasta) # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db


# 2. 查找数据
print('step 2: find existing data')
find_data = input_df.merge(latest_sprot, on='seq', how='left')
find_data = latest_sprot[latest_sprot.seq.isin(input_df.seq)]
find_data = find_data.drop_duplicates(subset='seq').reset_index(drop=True)
exist_data = find_data.merge(input_df, on='seq', how='left').iloc[:,np.r_[8,0,1:8]].rename(columns={'id_x':'uniprot_id','id_y':'input_id'}).reset_index(drop=True)
noExist_data = input_df[~input_df.seq.isin(find_data.seq)]

if len(noExist_data) == 0:
    exist_data=exist_data[['input_id','ec_number']].rename(columns={'ec_number':'ec_pred'})
    exist_data.to_csv(output_tsv, sep='\t', index=False)
    end = time.process_time()
    print('All done running time: %s Seconds'%(end-start))

step 1: loading data
step 2: find existing data


In [16]:
# 3. EMBedding
print('step 3: Embedding')
featurebank_esm32 = pd.read_feather(cfg.FILE_FEATURE_ESM32)

existing_feature = featurebank_esm32[featurebank_esm32.id.isin(exist_data.uniprot_id)]
existing_feature = exist_data[['input_id','uniprot_id']].merge(existing_feature.rename(columns={'id':'uniprot_id'}), on='uniprot_id', how='left').rename(columns={'input_id':'id'}).iloc[:,np.r_[0,2:existing_feature.shape[1]+1]]

rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=noExist_data, model='esm1b_t33_650M_UR50S',seqthres=1022)

rep32 = pd.concat([existing_feature,rep32],axis=0).reset_index(drop=True)

step 3: Embedding


100%|██████████| 51/51 [00:19<00:00,  2.58it/s]


In [31]:

# 4. sequence alignment
print('step 4: sequence alignment')
if not os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)
blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)
blast_res =blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2,3:10]].rename(columns={'id_x':'input_id','id_y':'uniprot_id'}).reset_index(drop=True)

# 5. isEnzyme Prediction
print('step 5: predict isEnzyme')
pred_dmlf = pd.DataFrame(rep32.id.copy())
model_isEnzyme = load_model(cfg.ISENZYME_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_isEnzyme.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t1=joblib.load(cfg.DICT_LABEL_T1)
pred_dmlf['dmlf_isEnzyme']=(encoder_t1.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]

# 6. How many Prediction
print('step 6: predict function counts')
model_howmany = load_model(cfg.HOWMANY_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_howmany.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t2=joblib.load(cfg.DICT_LABEL_T2)
pred_dmlf['dmlf_functions']=(encoder_t2.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]


# 7. EC Prediction
print('step 7: predict EC')
model_ec = load_model(cfg.EC_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t3=joblib.load(cfg.DICT_LABEL_T3)
pred_dmlf['dmlf_ec']=[','.join(item) for item in (encoder_t3.inverse_transform(bcommon.props_to_onehot(predicted)))]
pred_dmlf['dmlf_recomendations']=pd.DataFrame(predicted).apply(lambda x :sorted(dict(zip((encoder_t3.classes_), x)).items(),key = lambda x:x[1], reverse = True)[0:20], axis=1 ).values
pred_dmlf = pred_dmlf.merge(blast_res[['input_id','ec_number']].rename(columns={'ec_number':'blast_ec'}), left_on='id', right_on='input_id', how='left')
pred_dmlf['dmlf_recomendations']=pred_dmlf.apply(lambda x: x.dmlf_recomendations if x.dmlf_isEnzyme else '-', axis=1 )
pred_dmlf['dmlf_ec']=pred_dmlf.apply(lambda x: x.dmlf_ec if x.dmlf_isEnzyme else '-', axis=1 )

pred_dmlf = pred_dmlf.merge(exist_data[['input_id','ec_number']].rename(columns={'ec_number':'db_ec'}), on='input_id', how='left')
pred_dmlf['dmlf_ec']=pred_dmlf.apply(lambda x: x.db_ec if str(x.db_ec)!='nan' else x.dmlf_ec,axis=1)
output_df = pred_dmlf[['id', 'dmlf_isEnzyme', 'dmlf_functions', 'dmlf_ec', 'dmlf_recomendations', 'blast_ec' ]].rename(columns={'id':'input_id'})

step 4: sequence alignment
Write finished
diamond blastp -d /home/shizhenkun/codebase/ECRECer/data/uniprot_blast_db/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
step 5: predict isEnzyme
144/144 [==============================] - 1s 1ms/step
step 6: predict function counts


/home/shizhenkun/miniconda3/envs/DMLF/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


144/144 [==============================] - 1s 1ms/step
step 7: predict EC
144/144 [==============================] - 1s 1ms/step


In [35]:
pred_dmlf['dmlf_ec']=pred_dmlf.apply(lambda x: x.dmlf_ec if x.dmlf_isEnzyme else '-', axis=1 )


,input_id,dmlf_isEnzyme,dmlf_functions,dmlf_ec,dmlf_recomendations,blast_ec
0,sp|P0AG67|RS1_ECOLI,False,1,-,-,-
1,sp|P0A7U3|RS19_ECOLI,False,1,-,-,-
2,sp|P03813|YGEA_ECOLI,True,1,5.1.1.10,"[(5.-.-.-, 0.4982529282569885), (5.1.1.3, 0.27...",5.1.1.10
3,sp|P00935|METB_ECOLI,True,1,2.5.1.48,"[(2.5.1.48, 0.9103495478630066), (4.4.1.1, 0.0...",2.5.1.48
4,sp|P63284|CLPB_ECOLI,False,1,-,-,-
...,...,...,...,...,...,...
4584,tr|Q47042|Q47042_ECOLI,False,1,2.1.1.63,-,NaN
4585,tr|Q47556|Q47556_ECOLI,False,1,2.1.1.56,-,NaN
4586,tr|Q57049|Q57049_ECOLI,False,1,2.3.1.12,-,NaN
4587,tr|Q5H778|Q5H778_ECOLI,False,1,3.6.3.-,-,-


In [32]:
pred_dmlf

,id,dmlf_isEnzyme,dmlf_functions,dmlf_ec,dmlf_recomendations,input_id,blast_ec,db_ec
0,sp|P0AG67|RS1_ECOLI,False,1,-,-,sp|P0AG67|RS1_ECOLI,-,-
1,sp|P0A7U3|RS19_ECOLI,False,1,-,-,sp|P0A7U3|RS19_ECOLI,-,-
2,sp|P03813|YGEA_ECOLI,True,1,5.1.1.10,"[(5.-.-.-, 0.4982529282569885), (5.1.1.3, 0.27...",sp|P03813|YGEA_ECOLI,5.1.1.10,5.1.1.10
3,sp|P00935|METB_ECOLI,True,1,2.5.1.48,"[(2.5.1.48, 0.9103495478630066), (4.4.1.1, 0.0...",sp|P00935|METB_ECOLI,2.5.1.48,2.5.1.48
4,sp|P63284|CLPB_ECOLI,False,1,-,-,sp|P63284|CLPB_ECOLI,-,-
...,...,...,...,...,...,...,...,...
4584,tr|Q47042|Q47042_ECOLI,False,1,2.1.1.63,-,NaN,NaN,NaN
4585,tr|Q47556|Q47556_ECOLI,False,1,2.1.1.56,-,NaN,NaN,NaN
4586,tr|Q57049|Q57049_ECOLI,False,1,2.3.1.12,-,NaN,NaN,NaN
4587,tr|Q5H778|Q5H778_ECOLI,False,1,3.6.3.-,-,tr|Q5H778|Q5H778_ECOLI,-,NaN


In [9]:




    
    


    print('step 4: run prediction')

    if mode=='p':

        # 5. isEnzyme Prediction
        print('step 5: predict isEnzyme')
        pred_dmlf = pd.DataFrame(rep32.id.copy())
        model_isEnzyme = load_model(cfg.ISENZYME_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_isEnzyme.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t1=joblib.load(cfg.DICT_LABEL_T1)
        
        pred_dmlf['dmlf_isEnzyme']=(encoder_t1.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]


        # 6. How many Prediction
        print('step 6: predict function counts')
        model_howmany = load_model(cfg.HOWMANY_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_howmany.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t2=joblib.load(cfg.DICT_LABEL_T2)
        pred_dmlf['dmlf_howmany']=(encoder_t2.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]


        # 7. EC Prediction
        print('step 7: predict EC')
        model_ec = load_model(cfg.EC_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t3=joblib.load(cfg.DICT_LABEL_T3)
        pred_dmlf['dmlf_ec']=[','.join(item) for item in (encoder_t3.inverse_transform(bcommon.props_to_onehot(predicted)))]


        print('step 8: integrate results')
        results = pred_dmlf.merge(exist_data, left_on='id',right_on='input_id', how='left') 
        results=results.fillna('#')
        results['ec_pred'] =results.apply(lambda x : x.ec_number if x.ec_number!='#' else ('-' if x.dmlf_isEnzyme==False else x.dmlf_ec) ,axis=1)
        output_df = results[['id', 'ec_pred']].rename(columns={'id':'id_input'})

    elif mode =='r':
        print('step 4: recommendation')
        label_model_ec = pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()
        model_ec = load_model(f'{cfg.MODELDIR}/task3_esm32_2022.h5',custom_objects={"Attention": Attention}, compile=False)
        predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        output_df=pd.DataFrame()
        output_df['id']=input_df['id'].copy()
        output_df['ec_recomendations']=pd.DataFrame(predicted).apply(lambda x :sorted(dict(zip((label_model_ec), x)).items(),key = lambda x:x[1], reverse = True)[0:topnum], axis=1 ).values

    
    elif mode =='h':
        print('running in hybird mode')

        # 4. sequence alignment
        print('step 4: sequence alignment')
        if not os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
            funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
            cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
            os.system(cmd)
        blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)
        blast_res =blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2,3:10]].rename(columns={'id_x':'input_id','id_y':'uniprot_id'}).reset_index(drop=True)

        # 5. isEnzyme Prediction
        print('step 5: predict isEnzyme')
        pred_dmlf = pd.DataFrame(rep32.id.copy())
        model_isEnzyme = load_model(cfg.ISENZYME_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_isEnzyme.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t1=joblib.load(cfg.DICT_LABEL_T1)
        pred_dmlf['dmlf_isEnzyme']=(encoder_t1.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]


        # 6. How many Prediction
        print('step 6: predict function counts')
        model_howmany = load_model(cfg.HOWMANY_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_howmany.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t2=joblib.load(cfg.DICT_LABEL_T2)
        pred_dmlf['dmlf_functions']=(encoder_t2.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]


        # 7. EC Prediction
        print('step 7: predict EC')
        model_ec = load_model(cfg.EC_MODEL,custom_objects={"Attention": Attention}, compile=False)
        predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
        encoder_t3=joblib.load(cfg.DICT_LABEL_T3)
        pred_dmlf['dmlf_ec']=[','.join(item) for item in (encoder_t3.inverse_transform(bcommon.props_to_onehot(predicted)))]
        pred_dmlf['dmlf_recomendations']=pd.DataFrame(predicted).apply(lambda x :sorted(dict(zip((encoder_t3.classes_), x)).items(),key = lambda x:x[1], reverse = True)[0:20], axis=1 ).values

        pred_dmlf = pred_dmlf.merge(blast_res[['input_id','ec_number']].rename(columns={'ec_number':'blast_ec'}), left_on='id', right_on='input_id', how='left')
        pred_dmlf['dmlf_recomendations']=pred_dmlf.apply(lambda x: x.dmlf_recomendations if x.dmlf_isEnzyme else '-', axis=1 )
        pred_dmlf = pred_dmlf.merge(exist_data[['input_id','ec_number']].rename(columns={'ec_number':'db_ec'}), on='input_id', how='left')
        pred_dmlf['dmlf_ec']=pred_dmlf.apply(lambda x: x.db_ec if str(x.db_ec)!='nan' else x.dmlf_ec,axis=1)
        output_df = pred_dmlf[['input_id', 'dmlf_isEnzyme', 'dmlf_functions', 'dmlf_ec', 'dmlf_recomendations', 'blast_ec' ]]

    else:
        print(f'mode:{mode} not found')
        sys.exit()


    print('step 9: writting results') 

    output_df.to_csv(output_tsv, sep='\t', index=False)

    print(output_df)
    
    end = time.process_time()
    print('All done running time: %s Seconds'%(end-start))